# Cart Pole


# Dependencies

- pip install tensorflow==2.3.0
- pip install gym==0.17.2
- pip install keras==2.4.3
- pip install keras-rl2==1.0.4
- pip install pygame

Or as a one liner:
pip install tensorflow==2.3.0 gym==0.17.2 keras==2.4.3 keras-rl2==1.0.4 pygame

# Test environment (can be any) from OpenAI

In [2]:
import gym 
import random

In [3]:
env = gym.make('CartPole-v1', render_mode="human")
states = env.observation_space.shape[0]
actions = env.action_space.n

In [4]:
actions

2

In [5]:
states

4

In [6]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0,1])
        #n_state, reward, done, info = env.step(action)
        step_result = env.step(action)
        n_state = step_result[0]
        reward = step_result[1]
        done = step_result[2]
        info = step_result[3]
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:11.0
Episode:2 Score:12.0
Episode:3 Score:13.0
Episode:4 Score:20.0
Episode:5 Score:23.0
Episode:6 Score:27.0
Episode:7 Score:9.0
Episode:8 Score:20.0
Episode:9 Score:31.0
Episode:10 Score:10.0


Create Deep Learning model (with Keras)

In [14]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [15]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,actions))) # 1,2 is the input_shape, because there are 2 actions
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [16]:
model = build_model(states, actions)

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 2)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                72        
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 50        
Total params: 722
Trainable params: 722
Non-trainable params: 0
_________________________________________________________________


# Build Agent with Keras-RL

In [18]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [19]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [20]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


ValueError: setting an array element with a sequence.

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

In [ ]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

# Reloading Agent from Memory

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [ ]:
del model
del dqn
del env

In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)